### Some Important Terminologies:

* <u>*Nodes*</u> : These are the python functions that performs some task.

* <u>*Edges*</u> : These are the connection between nodes. They also provide the flow of information from one node to other.Basically, they maintain the direction of the data flow.

        There is also a special edge named "Conditional Edges".

* <u>*State*</u> : ```State``` a.k.a ```State Schema``` serves as the input for all nodes and edges in the graph. Here, ```State Schema``` means nothing but having properties or information, and these are given as input to all nodes and edges.

* <u>*State Graph*</u> : This represents the structure of entire graph.

---
## Building Simple workflow or Graph using LangGraph

### More about State:

* The State Schema serves as the input for all nodes and edges in the graph.

* Lets use the ```TypeDict``` class from python's typing module as our schema, which provides type hints for the keys.

In [1]:
from typing_extensions import TypedDict
class State(TypedDict):
    graph_info:str

### Nodes:
* These are the python functions that performs some task.


* The first positional argument is the state, as defined above.

* Because the state is a TypeDict with schema as defined above, each node can access the key, graph_state, with state['graph_state].

* Each node returns a new value of the state key graph_state.

* By default, the new value returned by each node will override the prior state value.

In [36]:
def start_play(state:State ):
    print("'start_play' node has been called.")
    # print(state["graph_info"])
    return {'graph_info':state['graph_info']  + "I am planning to play. "}

def badminton(state:State):
    print("Badminton node has been called")
    return {'graph_info':state['graph_info']  + "I will play the Badminton. "}

def cricket(state:State):
    print("Cricket node has been called")
    return {'graph_info':state['graph_info']  + "I will play the Cricket. "}

In [22]:
import random
from typing import Literal

def random_play(state:State) -> Literal['cricket','badminton']:
    graph_info = state['graph_info']
    if random.random()>0.5:
        return "cricket"
    else:
        return "badminton"

### Graph Construction:

* Now, we build the graph from our components defined above.

* The StateGraph class is the graph class that we can use.

* First, we initialize a StateGraph with the State class we defined above.

* Then, we add our nodes and edges.

* We use the `START` Node, a special node that sends user input to the graph, to indicate where to start our graph.

* The `END` Node is a special node that represents a terminal node.

* Finally, we compile our graph to perform a few basic checks on the graph structure.

* We can visualize the graph as a <u>Mermaid diagram</u>.

In [37]:
from IPython.display import display, Image
from langgraph.graph import StateGraph, START, END

## Build Graph
graph = StateGraph(state_schema=State)

## Adding the nodes
graph.add_node("start_play",start_play)
graph.add_node("cricket", cricket)
graph.add_node("badminton", badminton)

## Schedule the flow of the graph
graph.add_edge(START,"start_play")
graph.add_conditional_edges("start_play",random_play)
graph.add_edge("cricket",END)
graph.add_edge("badminton",END)

## Compile the graph
graph_builder = graph.compile()

## View the graph
# display(Image(graph_builder.get_graph().draw_mermaid_png())) # Can't view the graph if you are offline

In [42]:
graph_builder.invoke(
    {
        "graph_info":"Hey, My name is Kalyan. "
    }
)

'start_play' node has been called.
Cricket node has been called


{'graph_info': 'Hey, My name is Kalyan. I am planning to play. I will play the Cricket. '}